In [64]:
import random
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import deque

# Constants
WALL = '#'
PATH = '.'
START = 'S'
EXIT = 'E'
DISTANCE = 16
Rooms = []

# Room Types and their Colors
ROOM_TYPES = {
    "Prison": {"symbol": "E", "color": "Grey", "rgb": (80, 80, 80)},
    "Treasure Room": {"symbol": "T", "color": "Gold", "rgb": (255, 215, 0)},
    "Trap Room": {"symbol": "X", "color": "Orange", "rgb": (255, 140, 0)},
    "Monster Lair": {"symbol": "M", "color": "Blue", "rgb": (30, 144, 255)},
    "Secret Room": {"symbol": "S", "color": "Pink", "rgb": (255, 105, 180)},
    "Armory": {"symbol": "A", "color": "Silver", "rgb": (192, 192, 192)},
    "Library": {"symbol": "B", "color": "Brown", "rgb": (139, 69, 19)},
    "Laboratory": {"symbol": "L", "color": "Dark Green", "rgb": (0, 100, 0)},
    "Puzzle Room": {"symbol": "Z", "color": "Cyan", "rgb": (0, 255, 255)},
    "Magic Chamber": {"symbol": "C", "color": "Purple", "rgb": (138, 43, 226)}
}
# Room Descriptions - 50 items each
ADJECTIVES = [
    "dimly lit", "ancient", "dusty", "ornate", "mystical", "shimmering", "overgrown", "crumbling", "icy", "cursed",
    "hot", "colossal", "glamorous", "plain", "unkempt", "dazzling", "repulsive", "small", "tall", "lethal", 
    "fabulous", "sparkly", "peaceful", "ugly", "bright", "odd", "narrow", "jagged", "magical", "majestic", 
    "tense", "sad", "windy", "pristine", "rancid", "enchanting", "filthy", "glorious", "flat", "decayed", 
    "murky", "gloomy", "eerie", "smoky", "glistening", "sacred", "foul-smelling", "charred", "echoing", "ominous"
]

FEATURES = [
    "runes", "artifacts", "statues", "faint glows", "whispers", "cobwebs", "skeletons", "strange symbols",
    "glowing crystals", "rotting books", "twisting vines", "cracked mirrors", "floating candles", "silver chalices",
    "stone tablets", "gargoyle carvings", "bloodstains", "piles of bones", "enchanted tomes", "golden relics",
    "iron chains", "ancient writings", "piles of gold", "cursed paintings", "moss-covered walls", "shattered glass",
    "twisted roots", "burning torches", "glowing fungi", "dark portals", "rusted weapons", "torn banners", 
    "scattered scrolls", "moonlit inscriptions", "arcane sigils", "hollow statues", "withered vines", "old maps",
    "luminescent glyphs", "hidden doors", "skulls", "haunted dolls", "floating feathers", "corroded shields",
    "blackened cauldrons", "jade ornaments", "phantom imprints", "strange artifacts", "shadowy figures", "mystic orbs"
]

SOUNDS = [
    "a distant echo", "a low hum", "mysterious whispers", "a crackling fire", "dripping water",
    "a soft rustling", "a deep growl", "chanting voices", "faint music", "a sharp screech",
    "rattling chains", "an eerie silence", "a sudden gust of wind", "soft laughter", "a heartbeat-like thumping",
    "footsteps in the distance", "a hollow whisper", "a loud bang", "crackling static", "soft breathing",
    "a deep moan", "a spectral howl", "clinking metal", "soft scratching", "an otherworldly chime",
    "a pulsating vibration", "a distant explosion", "a metallic ringing", "a guttural growl", "a distant scream",
    "a single dripping noise", "a faint ticking", "muffled voices", "a sudden crash", "grinding stone",
    "a beastly snarl", "scraping wood", "a soft thud", "a chorus of whispers", "gasping breaths",
    "a ghostly moan", "drumming fingers", "a rhythmic tapping", "a low drone", "a mystical chime",
    "a gust of wind through cracks", "fluttering wings", "scratching from behind the walls", "a hollow knock", "echoing footsteps"
]

ACTIONS = [
    "inviting you in", "sending chills down your spine", "filling the air with unease", "making you feel watched",
    "urging you to move forward", "making the air feel heavy", "bringing a sense of nostalgia", "whispering unintelligible secrets",
    "creating an eerie tension", "pulling you toward the center", "giving an overwhelming feeling of dread", "making the walls seem alive",
    "distorting the space around you", "tempting you to explore", "pushing you back with an unseen force", "draining the warmth from your body",
    "making your skin tingle", "causing your vision to blur", "making it hard to breathe", "surrounding you with an unseen presence",
    "making your heartbeat race", "clouding your mind", "provoking an unexplained sorrow", "making the floor feel unstable",
    "enveloping you in a strange warmth", "mimicking voices you recognize", "making the air hum with energy", "repeating your footsteps behind you",
    "causing the light to flicker", "making shadows move on their own", "giving off a magnetic pull", "causing a strange ringing in your ears",
    "making time seem to slow down", "erasing the sound of your footsteps", "intensifying your fear", "pressing a weight upon your shoulders",
    "giving you the sensation of falling", "sending a tingling sensation through your body", "making it feel like you are being followed",
    "stirring up old memories", "causing an inexplicable chill", "making you feel both welcomed and threatened", "casting flickering shadows on the walls",
    "leaving a metallic taste in your mouth", "causing your surroundings to vibrate slightly", "creating a forceful energy in the room",
    "tricking your mind into hearing distant voices", "making the ground feel uneven", "making you feel strangely at peace",
    "giving the sense that something is hiding nearby", "making you feel lost in time"
]


SENTENCE_STRUCTURES = [
    "You go into a {adjective} chamber. {feature} line the walls, while {sound} fills the air, {action}.",
    "This {adjective} hall is filled with {feature}. The sound of {sound} echoes around you, {action}.",
    "A/An {adjective} passage stretches before you, adorned with {feature}. The air is thick with {sound}, {action}.",
    "You find yourself in a {adjective} room where {feature} stand ominously. {sound} can be heard, {action}.",
    "A/An unsettling, {adjective} aura lingers in the air. {feature} surround the room, while {sound} fills the silence, {action}.",
    "As you enter the {adjective} room, {feature} cast eerie shadows along the walls. The presence of {sound} is overwhelming, {action}.",
    "The {adjective} corridor twists ahead, lined with {feature}. A faint echo of {sound} follows you, {action}.",
    "A/An {adjective} archway looms before you, leading into a space filled with {feature}. In the distance, {sound} resonates, {action}.",
    "The {adjective} chamber hums with energy. {feature} flicker in the dim light as {sound} reverberates through the space, {action}.",
    "You cautiously walk into a {adjective} vault. The floor is littered with {feature}, while {sound} lingers in the background, {action}.",
    "A/An {adjective}, {adjective} energy is in the air. {feature} surround the room, while {sound} echos in the distance, {action}.",
    "You quickly step into a {adjective} room. The floor is covered with {feature}, while {sound} grows louder, {action}.",
    "You stumble into a {adjective} room. It contains numerous {feature}, while {sound} follow closely behind you, {action}.",
    "This {adjective} area is hoarded with {feature}. The sound of {sound} stop as you enter, {action}.",




]

def generate_room_description():
    """Generates a room description."""
    structure = random.choice(SENTENCE_STRUCTURES)
    return structure.format(
        adjective=random.choice(ADJECTIVES),
        feature=random.choice(FEATURES),
        sound=random.choice(SOUNDS),
        action=random.choice(ACTIONS)
    )


# UI Output widget for Jupyter
output = widgets.Output()

class Room:
    def __init__(self, x, y, width, height, room_type=None):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.room_type = room_type if room_type else random.choice(list(ROOM_TYPES.keys()))
        self.symbol = ROOM_TYPES[self.room_type]["symbol"]
        self.color = ROOM_TYPES[self.room_type]["rgb"]
        self.color_name = ROOM_TYPES[self.room_type]["color"]

    def place(self, grid, room_grid):
        """Places the room onto the grid and assigns colors."""
        for i in range(self.y, self.y + self.height):
            for j in range(self.x, self.x + self.width):
                if 0 <= i < len(grid) and 0 <= j < len(grid[0]):  
                    grid[i][j] = self.symbol  
                    room_grid[i][j] = self.room_type  
        return grid, room_grid

# Ensure Room Diversity when room count > 7 and then allows duplicates after 11 rooms

def generateRandomRooms(grid, room_grid, count, desc_list, max_room_size=3):
    room_types = list(ROOM_TYPES.keys())
    random.shuffle(room_types)

    height, width = len(grid), len(grid[0])
    exit_x, exit_y = width - 2, height - 2  

    def is_near_exit(x, y, w, h):
        """Ensure rooms don't touch the exit."""
        for i in range(y, y + h):
            for j in range(x, x + w):
                if abs(i - exit_y) <= 1 and abs(j - exit_x) <= 1:
                    return True
        return False  

    selected_rooms = []

    # If count > 7, ensure all room types are unique before repeating
    if count >= 11:
        selected_rooms = room_types[:]
        selected_rooms += random.choices(room_types, k=count - len(room_types))
    elif count > 7:
        selected_rooms = random.sample(room_types, count)
    else:
        selected_rooms = random.choices(room_types, k=count)

    for room_type in selected_rooms:
        for _ in range(10):  
            width = random.randint(2, max_room_size)
            height = random.randint(2, max_room_size)
            x = random.randint(1, len(grid[0]) - max_room_size - 1)
            y = random.randint(1, len(grid) - max_room_size - 1)

            if not is_near_exit(x, y, width, height):
                room = Room(x, y, width, height, room_type)
                room.place(grid, room_grid)

                # Always generate a new description per room 
                new_description = generate_room_description()

                # Append every room separately so even same colors have unique text
                desc_list.append((ROOM_TYPES[room_type]["symbol"], room_type, ROOM_TYPES[room_type]["color"], new_description))
                break  


# Generates the Maze
def create_maze(width, height, room_count, seed=None):
    if seed is None:
        seed = random.random()  
    random.seed(seed) 
    

    while True:
        maze = [[WALL for _ in range(width)] for _ in range(height)]
        room_grid = [[None for _ in range(width)] for _ in range(height)]
        desc_list = []  # Initialize description list

        def carve_passages_from(x, y):
            directions = [(0, 2), (2, 0), (0, -2), (-2, 0)]
            random.shuffle(directions)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 < nx < width - 1 and 0 < ny < height - 1 and maze[ny][nx] == WALL:
                    maze[y + dy // 2][x + dx // 2] = PATH  
                    maze[ny][nx] = PATH  
                    carve_passages_from(nx, ny)

        maze[1][1] = START  
        maze[0][1] = PATH  
        carve_passages_from(1, 1)

        maze[height - 3][width - 2] = EXIT  
        maze[height - 2][width - 3] = PATH  

        generateRandomRooms(maze, room_grid, room_count, desc_list)

        return maze, room_grid, desc_list, seed


# Displays Maze
def display_maze_graph(maze, room_grid):
    height, width = len(maze), len(maze[0])
    image = np.zeros((height, width, 3), dtype=np.uint8)

    for y in range(height):
        for x in range(width):
            if room_grid[y][x] in ROOM_TYPES:
                image[y, x] = ROOM_TYPES[room_grid[y][x]]["rgb"]
            elif maze[y][x] == START:
                image[y, x] = (0, 255, 0) #green 
            elif maze[y][x] == EXIT:
                image[y, x] = (255, 0, 0)  #red
            elif maze[y][x] == WALL:
                image[y, x] = (0, 0, 0)  #black
            else:
                image[y, x] = (255, 255, 255)  #white

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    plt.show()

# Generate and Display Maze
def generate_and_display_maze(*args):
    width = int(width_input.value)
    height = int(height_input.value)
    room_count = int(room_input.value)
    seed = float(seed_input.value) if seed_input.value.strip() else None  

    maze, room_grid, desc_list, new_seed = create_maze(width, height, room_count, seed)

    with output:
        clear_output(wait=True)
        print(f"Seed: {new_seed} - Use this to regenerate the same maze!\n")
        display_maze_graph(maze, room_grid)
        # Displaying Room Descriptions
        print("\n🔹 Room Descriptions:")
        if desc_list:
            descriptions_by_color = {}
        
            for symbol, room_type, color, description in desc_list:
                if (color, room_type) not in descriptions_by_color:
                    descriptions_by_color[(color, room_type)] = []
                descriptions_by_color[(color, room_type)].append(description)
        
            # Print descriptions with corresponding color names and room type
            for (color, room_type), descriptions in descriptions_by_color.items():
                color_symbol = {
                    "Gold": "🟨", "Brown": "🟫", "Orange": "🟧", "Grey": "⬜",
                    "Silver": "⬜", "Pink": "🟪", "Dark Green": "🟩",
                    "Blue": "🟦", "Purple": "🟪"
                }.get(color, "⬜")  # Default to ⬜ if color not found
        
                print(f"\n{color_symbol} {color} ({room_type})")  # ✅ Includes Room Type
                for desc in descriptions:
                    print(f"- {desc}")

        
        else:
            print("⚠️ No rooms were placed.")

# UI Elements
width_input = widgets.Text(value="30", description="Width:") # width automatically set to 30
height_input = widgets.Text(value="30", description="Height:") # height automatically set to 30
room_input = widgets.Text(value="10", description="Rooms:") #room number automatically set to 10
seed_input = widgets.Text(value="", description="Seed (Optional):")  #allows user to input maze seed number to return to one already displayed
generate_button = widgets.Button(description="Generate Maze") #click to get a new maze each time

display(width_input, height_input, room_input, seed_input, generate_button, output)
generate_button.on_click(generate_and_display_maze)
generate_and_display_maze()


Text(value='30', description='Width:')

Text(value='30', description='Height:')

Text(value='10', description='Rooms:')

Text(value='', description='Seed (Optional):')

Button(description='Generate Maze', style=ButtonStyle())

Output()